# 1. Loading packages

In [1]:
# Loading packages
import os
import pandas as pd
# !pip install sparse-dot-topn
import sparse_dot_topn
import sparse_dot_topn.sparse_dot_topn as ct
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import scipy
from scipy.sparse import csr_matrix
import time
import sys
import warnings
warnings.filterwarnings("ignore")
import multiprocessing
from fuzzywuzzy import fuzz, process
import tqdm
from tqdm import tqdm
from joblib import Parallel, delayed


# 2. Input parameters

In [2]:
## fast string matching using tf-idf, ngrams and cosine 
current_dir = "/Users/paritoshgupta/Downloads/"
input_file_name = "apr_10_null_state_entries_clusters.csv"
name_col = "LNAME_OR_CORPNAME"
primary_key = "SUB_ENTITY_ID"
cols_to_filter = ['SUB_ENTITY_ID','LNAME_OR_CORPNAME']
cosine_sim_lower_cutoff = 0.99
batch_size = 250000
output_seed_file_name = "apr12_seed_data_null_states.csv"

## Fuzzy Wuzzy
name_matching_col = "LNAME_OR_CORPNAME"
lower_cutoff_name_matching = 95
output_fuzzy_match_file_name = "apr12_fuzzy_match_null_states.csv"

# 3. UDFs

In [3]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'InteractorA': left_side,
                          'InteractorB': right_side,
                           'Cosine_Similairity': similairity})


def fast_string_matching_batch(data,name_col,cossim_lower_cutoff, batch_size):
    print("############## Starting Fast String Matching... ############## ")
    start_func = time.time()
    big_frame = pd.DataFrame()
    data = data.sort_values(name_col,ascending=True).reset_index(drop=True)
    print("Raw Input Data Size: ", data.shape)
    print(""); print("")
    n_rows = data.shape[0]
    for i in range(0,n_rows,batch_size):
        if i + batch_size <= n_rows:
            print("# Fast String Matching on batch of data with row number - " + 
                  str(i) + " to " + str(i + batch_size) +"  #")
            df_sample = data[i:i+batch_size]
        else:
            df_sample = data[i:]  
            print("# Fast String Matching on batch of data with row number - " + str(i) + " to " + str(n_rows) +
                             "  #")
        company_names = df_sample[name_col].drop_duplicates().reset_index(drop=True)
        print("Unique list size: ", str(len(company_names)))
        start = time.time()
        vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
        tf_idf_matrix = vectorizer.fit_transform(company_names)
        matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10000, cossim_lower_cutoff)
        matches_df = get_matches_df(matches, company_names, top=100000)
        # Remove all exact matches
        matches_df = matches_df[matches_df['Cosine_Similairity'] < 0.99999].drop_duplicates().reset_index(drop=True) 
        matches_df = matches_df.sort_values('Cosine_Similairity',ascending=False).reset_index(drop=True)
        print("Output shape: ", matches_df.shape)
        if len(matches_df) > 0: 
            matches_df['check_string'] = matches_df.apply(lambda row: ''.join(sorted([row['InteractorA'], 
                                                                                      row['InteractorB']])), axis=1)
            matches_df = matches_df.drop_duplicates('check_string').reset_index(drop=True)
            matches_df = matches_df.drop(['check_string'],axis=1)
            print("Output shape after dropping swapped duplicates: ", matches_df.shape)
            matches_df = matches_df[~matches_df['InteractorB'].isin(matches_df['InteractorA'])]
            big_frame = big_frame.append(matches_df, ignore_index=True)  
            print("Execution time of Fast String Matching on current batch: " + str(execution_time_parser(start)))
            print(""); print("")
    
    print("### Total Execution Time: " + str(execution_time_parser(start_func)) + "###")
    
    return big_frame
        
def melt_data_create_clusters(input_data,name_col):
    input_data['CLUSTER_ID'] = pd.Categorical(input_data['InteractorA'].astype(str)).codes
    melted_data = pd.melt(input_data, id_vars='CLUSTER_ID', 
                                 value_vars=['InteractorA','InteractorB'], \
                                 var_name=None, value_name=name_col, \
                                 col_level=None).drop('variable',axis=1).sort_values('CLUSTER_ID'). \
                                 reset_index(drop=True)[[name_col,'CLUSTER_ID']]
    return melted_data
    
def execution_time_parser(start):
    end = time.time()
    temp = end - start
    hours = temp // 3600
    temp = temp - 3600 * hours
    minutes = temp // 60
    seconds = temp - 60 * minutes
    time_taken = str(hours) + " hours " + str(minutes) + " minutes " + str(int(seconds)) + " seconds"

    return time_taken

def seed_delta_data_clean(data, type):
    try:
        if type == "seed":
            data.columns = ["SEED_" + x for x in data.columns]
        else:
            data.columns = ["DELTA_" + x for x in data.columns]
        cols_to_clean = data.select_dtypes(exclude=['int64', 'int', 'int32', 'float64', 'float']).columns

        for col in cols_to_clean:
            data[col] = data[col].map(lambda x: " ".join(x.strip().upper() for x in str(x).split()))

    except Exception as e:
        error = "Error in cleaning Seed and Delta data: " + str(e)
        print(error)
        sys.exit(1)

    return data

def seed_delta_matching_col_clean(input_text):
    try:
        input_text_mod = ' '.join([str(x) for x in input_text.lower().strip().split()])
    except Exception as e:
        error = "Error in cleaning matching columns of Seed and Delta: " + str(e)
        print(error)
        sys.exit(1)
    return input_text_mod

def iterator_fuzzy_score(text, dict_name, score_cutoff):
    try:
        tmp_tuple = process.extractOne(text, dict_name, score_cutoff=score_cutoff,
                                       scorer=fuzz.ratio)
        if tmp_tuple:
            tmp_tuple = tmp_tuple[1:]
        else:
            tmp_tuple = (0, np.NaN)
    except Exception as e:
        error = "Error in fuzzy score calculator: " + str(e)
        print(error)
        print(error)
        sys.exit(1)
    return tmp_tuple


def fuzzy_score_calculator_name(delta_data, seed_data, matching_col, primary_key, state_to_process, lower_cut_off):
    try:
        print("START TIME: " + str(pd.datetime.now()))
        print("Starting Name Fuzzy Matching for leftover Entities after Address matching for state: "
                     + str(state_to_process))
        print("SEED Data Size: " + str(len(seed_data)))
        print("DELTA Data Size: " + str(len(delta_data)))
        delta_list_name = [seed_delta_matching_col_clean(x) for x in
                           delta_data[str("DELTA_" + matching_col)].values.tolist()]
        seed_dict_name = dict(zip(seed_data[str("SEED_" + primary_key)], [seed_delta_matching_col_clean(x) for
                                                                          x in
                                                                          seed_data[str("SEED_" + matching_col)]]))

        num_cores = multiprocessing.cpu_count()
        print("Processing Fuzzy Matching on: " + str(num_cores) + " cores")
        tuple_list_name = Parallel(n_jobs=-1, backend="multiprocessing")(
            delayed(iterator_fuzzy_score)(delta_name, seed_dict_name,
                                          lower_cut_off) for delta_name in tqdm(delta_list_name))

        df_name_matching = pd.DataFrame(tuple_list_name, columns=['FUZZY_SCORE', str("SEED_" + primary_key)])
        df_out = pd.concat([delta_data, df_name_matching], axis=1)
        df_out = pd.merge(df_out, seed_data, on=str("SEED_" + primary_key), how="inner")
        df_out = df_out.drop(df_out[(df_out[str('DELTA_' + matching_col)] == "not provided") |
                                    (df_out[str('SEED_' + matching_col)] == "not provided")].index)
        df_out = df_out[df_out['FUZZY_SCORE'] >= lower_cut_off]
        df_out['FUZZY_MATCHING_TYPE'] = "NAME"
        print("Finished fuzzy matching on Name for state: " + str(state_to_process))
    except Exception as e:
        error = "Error in fuzzy matching on Name for state: " + str(state_to_process) + " " + str(e)
        print(error)
        sys.exit(1)

    return df_out


# 4. Data Reading and % distribution

In [4]:
os.chdir(current_dir)
data = pd.read_csv(input_file_name,low_memory=False)[cols_to_filter]
print("shape of input data: ", data.shape)
print("Columns present are:", list(data.columns))

# Missing % in Name column
print("\n% NULL in NAME:", 
           str(100 * (1 - (data[~data[name_col].isnull()].shape[0])/data.shape[0])))  


shape of input data:  (2179588, 2)
Columns present are: ['SUB_ENTITY_ID', 'LNAME_OR_CORPNAME']

% NULL in NAME: 0.0


# 5. Processing using TF-IDF, n-grams: sequences of N contiguous items (in this case characters), Cosine Similarity ...

In [16]:
bigframe = fast_string_matching_batch(data, name_col=name_col, 
                                      cossim_lower_cutoff=cosine_sim_lower_cutoff, batch_size=batch_size)
# bigframe.to_csv("null_states_seed_data_raw.csv",index=False)
seed_clusters = melt_data_create_clusters(bigframe,name_col)
seed_clusters.to_csv(output_seed_file_name,index=False)


############## Starting Fast String Matching... ############## 
Raw Input Data Size:  (2179588, 2)


# Fast String Matching on batch of data with row number - 0 to 250000  #
Unique list size:  236890
Output shape:  (118, 3)
Output shape after dropping swapped duplicates:  (59, 3)
Execution time of Fast String Matching on current batch: 0.0 hours 3.0 minutes 12 seconds


# Fast String Matching on batch of data with row number - 250000 to 500000  #
Unique list size:  231934
Output shape:  (338, 3)
Output shape after dropping swapped duplicates:  (169, 3)
Execution time of Fast String Matching on current batch: 0.0 hours 3.0 minutes 11 seconds


# Fast String Matching on batch of data with row number - 500000 to 750000  #
Unique list size:  237163
Output shape:  (208, 3)
Output shape after dropping swapped duplicates:  (104, 3)
Execution time of Fast String Matching on current batch: 0.0 hours 3.0 minutes 9 seconds


# Fast String Matching on batch of data with row number - 750000 to 1000

# 6. Creating seed and delta datasets

In [5]:
# seed_clusters = pd.read_csv("apr12_seed_data_null_states.csv")
seed_data = pd.merge(seed_clusters,data,on=name_col,how="inner").reset_index(drop=True)
delta_data = data[~data[primary_key].isin(seed_data[primary_key])].reset_index(drop=True)
seed_data_clean = seed_delta_data_clean(data=seed_data, type="seed")
delta_data_clean = seed_delta_data_clean(data=delta_data, type="delta") 
print("Columns present in SEED data: ", list(seed_data_clean.columns))
print("Columns present in DELTA data: ", list(delta_data_clean.columns))


Columns present in SEED data:  ['SEED_LNAME_OR_CORPNAME', 'SEED_CLUSTER_ID', 'SEED_SUB_ENTITY_ID']
Columns present in DELTA data:  ['DELTA_SUB_ENTITY_ID', 'DELTA_LNAME_OR_CORPNAME']


# 7. Fuzzy Name Matching on NAME column

In [6]:
tik = time.time()
# delta_data_clean = delta_data_clean[0:10000] (testing purpose)
df_out_fuzzy_matching_name = fuzzy_score_calculator_name(delta_data=delta_data_clean,
                                                             seed_data=seed_data_clean,
                                                             matching_col=name_matching_col,
                                                             primary_key=primary_key,
                                                             state_to_process="NULL STATES",
                                                             lower_cut_off=lower_cutoff_name_matching)
df_out_fuzzy_matching_name.to_csv(output_fuzzy_match_file_name,index=False)

print("######## Summary ########")
print("Output file name: ", str(output_fuzzy_match_file_name))
print("Output data shape: ", str(len(df_out_fuzzy_matching_name)))
print("##Total Execution Time: " +  str(execution_time_parser(tik)) + "##")

START TIME: 2019-04-12 15:21:52.594052
Starting Name Fuzzy Matching for leftover Entities after Address matching for state: NULL STATES
SEED Data Size: 2930
DELTA Data Size: 2176711


  0%|          | 0/2176711 [00:00<?, ?it/s]

Processing Fuzzy Matching on: 12 cores


100%|██████████| 2176711/2176711 [3:53:27<00:00, 155.40it/s]


Finished fuzzy matching on Name for state: NULL STATES
######## Summary ########
Output file name:  apr12_fuzzy_match_null_states.csv
Output data shape:  8248
##Total Execution Time: 3.0 hours 53.0 minutes 32 seconds##
